## 02 Data preprocessing

In this notebook we split the data into train and test subsets and discuss the validation.

### 1. Data loading and adjusting

We load the data and rename columns as mentioned in 01_exploratory_data_analysis.ipynb to standarize column names.

We know that there are 4 entries with wrong values, 1 for obesity and 3 for comorbidity. Obesity did not show significant correlation with the gallstone status. In addition, there are other metrics that likely contain similar biological information (e.g. BMI). We could leave the obesity outlier in the dataset but decided to keep our data as clean as possible. On the other hand, comorbidity is likely to be correlated with the gallstone status, we exclude from the dataset entries with comobidity above 1 (it is expected to be binary).


In [423]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold


path_train_data = "data/train_data_fold_split.csv"
path_train_data_strat = "data/train_data_fold_split_strat.csv"
path_test_data = "data/test_data.csv"
path_test_data_strat = "data/test_data_strat.csv"
save_files = True # change to True to rewrite train data and test data files

In [424]:
df_orig = pd.read_csv("data/gallstone_.csv")
col_map = {
    'Gallstone Status': 'gallstone',
    'Age': 'age',
    'Gender': 'gender',
    'Comorbidity': 'comorbidity',
    'Coronary Artery Disease (CAD)': 'cad',
    'Hypothyroidism': 'hypothyroidism',
    'Hyperlipidemia': 'hyperlipidemia',
    'Diabetes Mellitus (DM)': 'diabetes',
    'Height': 'height',
    'Weight': 'weight',
    'Body Mass Index (BMI)': 'bmi',
    'Total Body Water (TBW)': 'tbw',
    'Extracellular Water (ECW)': 'ecw',
    'Intracellular Water (ICW)': 'icw',
    'Extracellular Fluid/Total Body Water (ECF/TBW)': 'ecf_tbw',
    'Total Body Fat Ratio (TBFR) (%)': 'tbfr',
    'Lean Mass (LM) (%)': 'lm',
    'Body Protein Content (Protein) (%)': 'protein',
    'Visceral Fat Rating (VFR)': 'vfr',
    'Bone Mass (BM)': 'bm',
    'Muscle Mass (MM)': 'mm',
    'Obesity (%)': 'obesity',
    'Total Fat Content (TFC)': 'tfc',
    'Visceral Fat Area (VFA)': 'vfa',
    'Visceral Muscle Area (VMA) (Kg)': 'vma',
    'Hepatic Fat Accumulation (HFA)': 'hfa',
    'Glucose': 'glucose',
    'Total Cholesterol (TC)': 'tc',
    'Low Density Lipoprotein (LDL)': 'ldl',
    'High Density Lipoprotein (HDL)': 'hdl',
    'Triglyceride': 'triglyceride',
    'Aspartat Aminotransferaz (AST)': 'ast',
    'Alanin Aminotransferaz (ALT)': 'alt',
    'Alkaline Phosphatase (ALP)': 'alp',
    'Creatinine': 'creatinine',
    'Glomerular Filtration Rate (GFR)': 'gfr',
    'C-Reactive Protein (CRP)': 'crp',
    'Hemoglobin (HGB)': 'hgb',
    'Vitamin D': 'vitamin_d'
}

df = df_orig.rename(columns=col_map)

df = df[df["comorbidity"]<2]
df = df[df["obesity"]<1900]
df.reset_index(drop=True)

print(f"The dataset shape after excluding outliers: {df.shape}")

The dataset shape after excluding outliers: (315, 39)


### 2. Data Split Concept

We use k-fold cross-validation with repetitions (k=5, rep = 3) for both the regression and neural network models. Although k-fold is uncommon for large NNs, it is useful for our small dataset (315 samples) to get reliable performance estimates.

A hold-out test set (30%) is kept for final evaluation. EDA showed that gallstone tatus is not evenly distributed across genders. Here, to experimenz we propose two stratification strategies:
- stratification by gallstone status to ensure that the disease status is balanced in train and test datatsets. Such stratification normally produces more stable models;
- stratification by combination of gallstone status and gender to preserve joint distribution of gallstone and gender. In this way. likely the realistic patient distribution will be used for both sets. On the other hand, since the dataset is 315 samples only, such a stratification can hide natural variability in the data.

The same stratification is used for train/test and fold splitting 
Train/test files are saved separately, with the stratifiction column and k-fold assignments stored in the train file for reproducibility.

```
Full Dataset (315 samples)
│
├── 70% Train (220 samples) (train_data.csv) ──> K-Fold (k=5)
│      ├── Fold 1: Train / Validation
│      ├── Fold 2: Train / Validation
│      ├── Fold 3: Train / Validation
│      ├── Fold 4: Train / Validation
│      └── Fold 5: Train / Validation
│
└── 30% Test (95 samples) (test_data.csv) ──> Final evaluation
```

### 3. Data split to train and test sets.

There is an misbalance in gender distributions grouped by gallstone status. To take it into account we stratify the split by gender in addition to the gallstone status.

In [425]:
df["strat"] = df["gallstone"].astype(str) +  "_"  + df["gender"].astype(str)

train_data, test_data, y_train, y_test = train_test_split(df, df["gallstone"], test_size=0.3, stratify = df["gallstone"], random_state = 37)
train_data_strat, test_data_strat, y_train_strat, y_test_strat = train_test_split(df, df["gallstone"], test_size=0.3, stratify=df["strat"], random_state = 37)


Let's verify how the gender distribution looks like after random (stratified by gallstone status only) splitting and a splitting stratified by gallstone + gender.

In [426]:
def check_stratification_on_gallstone(df, feature, data_label = "train"):
    ct = pd.crosstab(df['gallstone'], df[feature])
    if feature == "gender":
        ct = ct.rename(columns = {0: "male", 1: "female"})
    ct_norm = ct.div(ct.sum(axis=1), axis=0).add_suffix(" (%)")* 100

    ct_side_by_side = pd.concat([ct, ct_norm.round(0).astype(int)], axis=1)

    display(ct_side_by_side.style.set_caption(f"{feature} distribution on gallstone status for {data_label} data"))

check_stratification_on_gallstone(df = train_data, feature = "gender", data_label="train")
check_stratification_on_gallstone(df = test_data, feature = "gender", data_label="test")

check_stratification_on_gallstone(df = train_data_strat, feature = "gender", data_label="train stratified")
check_stratification_on_gallstone(df = test_data_strat, feature = "gender", data_label="test stratified")



gender,male,female,male (%),female (%)
gallstone,,,,
0,63,47,57,43
1,52,58,47,53


gender,male,female,male (%),female (%)
gallstone,,,,
0,28,20,58,42
1,16,31,34,66


gender,male,female,male (%),female (%)
gallstone,,,,
0,64,47,58,42
1,47,62,43,57


gender,male,female,male (%),female (%)
gallstone,,,,
0,27,20,57,43
1,21,27,44,56


The gender distribution is the same for gender stratified train and test datasets. We can now proceed with k-fold splitting.

### 4. Train data splitting to k-folds

As described above, we use stratification by gallstone status + gender for k-folds. K equals 5.

In [427]:
# make stratified split and add columns "FoldXRepY" to df where values are "train" or "val"
# in such a way we can save splits to a file together with the data for better reproducebility

train_data.reset_index(drop = True, inplace = True)
train_data_strat.reset_index(drop = True, inplace = True)

n_splits = 5
n_repeats = 3
skf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats = n_repeats, random_state=42)

splits = {} # save splits for a check later
splits_strat = {} # save splits for a check later

for fold, (train_idx, val_idx) in enumerate(skf.split(train_data, train_data["gallstone"])):
    fold = fold
    fold_num = fold%n_splits + 1
    rep_num = fold//n_splits + 1
    fold_col = f"Fold{fold_num}_Rep{rep_num}"
    print(f"{fold_col}: Train {len(train_idx)}, Val {len(val_idx)}")
    train_data[fold_col] = "train"
    train_data.loc[val_idx, fold_col] = "val"
    splits[fold_col] =  {"train": train_idx.tolist(), "val": val_idx.tolist()} 

for fold, (train_idx, val_idx) in enumerate(skf.split(train_data, train_data_strat["strat"])):
    fold = fold
    fold_num = fold%n_splits + 1
    rep_num = fold//n_splits + 1
    fold_col = f"Fold{fold_num}_Rep{rep_num}"
    #print(f"{fold_col}: Train {len(train_idx)}, Val {len(val_idx)}")
    train_data_strat[fold_col] = "train"
    train_data_strat.loc[val_idx, fold_col] = "val"
    splits_strat[fold_col] =  {"train": train_idx.tolist(), "val": val_idx.tolist()} 

Fold1_Rep1: Train 176, Val 44
Fold2_Rep1: Train 176, Val 44
Fold3_Rep1: Train 176, Val 44
Fold4_Rep1: Train 176, Val 44
Fold5_Rep1: Train 176, Val 44
Fold1_Rep2: Train 176, Val 44
Fold2_Rep2: Train 176, Val 44
Fold3_Rep2: Train 176, Val 44
Fold4_Rep2: Train 176, Val 44
Fold5_Rep2: Train 176, Val 44
Fold1_Rep3: Train 176, Val 44
Fold2_Rep3: Train 176, Val 44
Fold3_Rep3: Train 176, Val 44
Fold4_Rep3: Train 176, Val 44
Fold5_Rep3: Train 176, Val 44


We can now save the train data to the file. Note that we keep gallstone status, stratification column and fold splittings together with the features for better tracebility.

In [428]:
# save train data with splits to a file

if save_files:
    train_data.to_csv(path_train_data, index = False)
    test_data.to_csv(path_test_data, index = False)
    train_data_strat.to_csv(path_train_data_strat, index = False)
    test_data_strat.to_csv(path_test_data_strat, index = False)

We quickly check how the saved data looks like.

In [429]:
train_data_loaded = pd.read_csv(path_train_data)
train_data_strat_loaded = pd.read_csv(path_train_data_strat)

display(train_data_loaded.head(10).style.format())

,gallstone,age,gender,comorbidity,cad,hypothyroidism,hyperlipidemia,diabetes,height,weight,bmi,tbw,ecw,icw,ecf_tbw,tbfr,lm,protein,vfr,bm,mm,obesity,tfc,vfa,vma,hfa,glucose,tc,ldl,hdl,triglyceride,ast,alt,alp,creatinine,gfr,crp,hgb,vitamin_d,strat,Fold1_Rep1,Fold2_Rep1,Fold3_Rep1,Fold4_Rep1,Fold5_Rep1,Fold1_Rep2,Fold2_Rep2,Fold3_Rep2,Fold4_Rep2,Fold5_Rep2,Fold1_Rep3,Fold2_Rep3,Fold3_Rep3,Fold4_Rep3,Fold5_Rep3
0,0,44,0,0,0,0,0,0,178,86.400000,27.300000,49.200000,18.700000,29.500000,40.000000,20.200000,79.750000,16.980000,8,3.400000,65.500000,24.000000,17.500000,10.600000,35.900000,2,103.000000,194.000000,127.000000,39.000000,180.000000,22.000000,40.000000,73.000000,0.820000,116.750000,0.000000,15.700000,25.150000,0_0,train,train,train,val,train,val,train,train,train,train,train,train,val,train,train
1,0,32,0,0,0,0,0,0,170,64.300000,22.200000,39.800000,16.500000,23.300000,41.000000,15.300000,84.760000,16.670000,4,2.700000,51.800000,1.100000,9.800000,5.900000,27.600000,1,91.000000,185.000000,125.000000,35.000000,165.000000,23.000000,22.000000,69.000000,0.670000,127.220000,0.500000,16.000000,22.925000,0_0,val,train,train,train,train,train,train,train,val,train,train,train,train,val,train
2,1,57,0,0,0,0,0,0,186,118.100000,34.100000,57.400000,22.000000,35.000000,38.300000,29.890000,70.110000,17.160000,16,2.000000,78.700000,33.870000,35.300000,21.630000,40.500000,2,83.000000,209.000000,139.000000,44.000000,148.000000,22.000000,33.000000,80.000000,0.640000,110.400000,0.500000,13.600000,18.700000,1_0,val,train,train,train,train,train,train,train,train,val,train,train,train,train,val
3,0,51,1,1,0,1,0,0,166,59.300000,21.500000,33.000000,14.100000,18.900000,43.000000,21.600000,78.410000,17.900000,4,2.400000,44.100000,2.100000,12.800000,5.200000,25.800000,0,99.000000,254.000000,158.000000,64.000000,107.000000,21.000000,21.000000,61.000000,0.700000,93.340000,0.000000,13.500000,46.900000,0_1,val,train,train,train,train,train,train,val,train,train,val,train,train,train,train
4,1,55,0,0,0,0,0,0,177,106.300000,33.900000,50.600000,21.900000,28.700000,43.000000,32.200000,67.830000,16.020000,18,3.600000,68.500000,54.300000,34.200000,30.800000,37.100000,0,94.500000,240.000000,148.000000,55.500000,141.500000,17.000000,15.000000,70.500000,0.715000,100.700000,0.650000,14.150000,17.750000,1_0,val,train,train,train,train,train,train,train,val,train,val,train,train,train,train
5,1,57,0,1,0,0,0,0,169,104.200000,36.500000,55.000000,23.000000,32.000000,41.810000,26.780000,73.220000,15.900000,16,3.800000,72.600000,43.070000,27.900000,16.550000,37.400000,3,98.000000,151.000000,95.000000,41.000000,121.000000,28.000000,48.000000,41.000000,0.830000,101.400000,0.000000,15.000000,5.700000,1_0,train,train,train,train,val,train,train,val,train,train,train,train,train,val,train
6,1,35,0,0,0,0,0,0,171,83.800000,28.700000,47.100000,19.000000,28.000000,40.330000,22.170000,77.830000,16.010000,8,2.300000,62.000000,22.090000,18.600000,10.880000,33.300000,2,94.000000,207.000000,139.000000,48.000000,108.000000,21.000000,39.000000,66.000000,0.850000,116.200000,8.100000,16.500000,11.200000,1_0,train,train,train,val,train,train,train,val,train,train,train,train,train,val,train
7,1,39,1,1,0,0,0,0,165,74.000000,27.200000,33.700000,15.100000,18.600000,45.000000,36.400000,63.600000,13.460000,6,2.400000,44.700000,23.500000,26.900000,13.100000,25.900000,0,103.000000,219.000000,136.000000,63.000000,94.000000,19.000000,19.000000,49.000000,0.580000,117.900000,0.000000,11.900000,9.500000,1_1,train,train,train,val,train,val,train,train,train,train,train,train,train,train,val
8,1,51,1,0,0,0,0,0,158,66.500000,26.600000,32.000000,12.000000,20.000000,37.500000,28.800000,71.130000,18.600000,6,2.400000,44.900000,8.730000,19.200000,8.100000,26.500000,0,96.000000,356.000000,262.000000,62.000000,79.000000,14.000000,15.000000,54.000000,0.750000,96.300000,9.000000,12.900000,21.100000,1_1,train,train,train,train,val,train,train,train,train,val,train,train,train,train,val
9,0,36,1,0,0,0,0,0,173,78.100

### 4 Extracting fold indices from the loaded data

We can now extract indices for train and validation set for each fold.


In [430]:
folds = train_data_loaded[[c for c in train_data_loaded.columns if c.startswith("Fold")]]
folds_strat = train_data_strat_loaded[[c for c in train_data_strat_loaded.columns if c.startswith("Fold")]]

splits_from_file = {} # to check if our splits are read correctly
splits_strat_from_file = {} 

for fold in folds.columns:
    train_idx =  folds.index[folds[fold] == "train"].to_numpy()
    val_idx = folds.index[folds[fold] == "val"].to_numpy()
    splits_from_file[fold] = {"train": train_idx.tolist(), "val": val_idx.tolist()}

for fold in folds_strat.columns:
    train_idx =  folds_strat.index[folds_strat[fold] == "train"].to_numpy()
    val_idx = folds_strat.index[folds_strat[fold] == "val"].to_numpy()
    splits_strat_from_file[fold] = {"train": train_idx.tolist(), "val": val_idx.tolist()}


A sanity check: verify if splits obtaind from StratifiedKFold in the notebook are identical to the splits loaded from the file.

In [431]:
print(f"Sanity check 1: are splits from RepeatedStratifiedKFold and from the train data file identical? {splits == splits_from_file}")
print(f"Sanity check 2: are splits_strat from RepeatedStratifiedKFold and from the train data strat file identical? {splits_strat == splits_strat_from_file}")
print(f"Sanity check 3: are splits and splits_strat identical? {splits == splits_strat}")

Sanity check 1: are splits from RepeatedStratifiedKFold and from the train data file identical? True
Sanity check 2: are splits_strat from RepeatedStratifiedKFold and from the train data strat file identical? True
Sanity check 3: are splits and splits_strat identical? False


### 5 Summary
In this notebook we 
- splitted the data to train and test sets using stratification by gallstone status and gender (30% test, 70% train i.e. 95 samples test, 220 samples train) 
- saved splits with fold information and the tratification column to files for better tracebility